In [1]:
# импорт необходимых библиотек
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# изпользование gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# загрузка данных
class CustomFashionMNISTDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape(28, 28)
        label = int(self.data.iloc[idx, 0])
        img = Image.fromarray(img).convert('RGB')  # преобразование в RGB
        if self.transform:
            img = self.transform(img)
        return img, label

In [4]:
# преобразование данных в тензоры
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# пути к файлам
train_csv_path = 'fashion-mnist_train.csv'
test_csv_path = 'fashion-mnist_test.csv'

In [5]:
# разделение на выборки
train_dataset = CustomFashionMNISTDataset(csv_file=train_csv_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = CustomFashionMNISTDataset(csv_file=test_csv_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# использование существующий предобученной модели resnet101 для улучшения результатов
model = models.resnet101(pretrained=True)

# замена выходного слоя
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10 классов в FashionMNIST

# перенос вычеслиний на GPU
model = model.to(device)

In [7]:
# оптимизация модели
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# процесс обчения модели
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

In [9]:
# процесс оценки модели
def evaluate_model(model, test_loader):
    model.eval()
    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
    
    accuracy = accuracy_score(true_labels, predicted_labels)
    print('Accuracy of the network on the test images: %.2f %%' % (accuracy * 100))

    return predicted_labels

In [10]:
# вычисление
train_model(model, criterion, optimizer, num_epochs=5)
predicted_labels = evaluate_model(model, test_loader)

[1,   100] loss: 0.705
[1,   200] loss: 0.458
[1,   300] loss: 0.393
[1,   400] loss: 0.342
[1,   500] loss: 0.409
[1,   600] loss: 0.574
[1,   700] loss: 0.421
[1,   800] loss: 0.393
[1,   900] loss: 0.342
[2,   100] loss: 0.331
[2,   200] loss: 0.455
[2,   300] loss: 0.323
[2,   400] loss: 0.310
[2,   500] loss: 0.302
[2,   600] loss: 0.286
[2,   700] loss: 0.309
[2,   800] loss: 0.355
[2,   900] loss: 0.282
[3,   100] loss: 0.239
[3,   200] loss: 0.251
[3,   300] loss: 0.253
[3,   400] loss: 0.274
[3,   500] loss: 0.271
[3,   600] loss: 0.351
[3,   700] loss: 0.291
[3,   800] loss: 0.299
[3,   900] loss: 0.249
[4,   100] loss: 0.222
[4,   200] loss: 0.238
[4,   300] loss: 0.253
[4,   400] loss: 0.254
[4,   500] loss: 0.240
[4,   600] loss: 0.242
[4,   700] loss: 0.226
[4,   800] loss: 0.234
[4,   900] loss: 0.242
[5,   100] loss: 0.211
[5,   200] loss: 0.220
[5,   300] loss: 0.206
[5,   400] loss: 0.216
[5,   500] loss: 0.230
[5,   600] loss: 0.219
[5,   700] loss: 0.215
[5,   800] 

In [11]:
# сохранение предсказанных значений в файл
ids = list(range(len(predicted_labels)))
submission_df = pd.DataFrame({'Id': ids, 'Category': predicted_labels})
submission_df.to_csv('submission.csv', index=False)